In [1]:
from datasets.get_datasets import *
from revision import *
from boostsrl import boostsrl
import random
from sklearn.model_selection import KFold

## Parameters

In [2]:
target = 'advisedby'
#test_size = 0.3
small_train_size = 0.3
validation_size = 0.2
revision_threshold = 0.8
max_revision_iterations = 10

## Dataset

Sameproject, sameperson and samecourse predicates are not being used.

In [3]:
[facts, pos, neg] = get_uwcse_dataset(target, acceptedPredicates=[
'professor',
'student',
'advisedby',
'tempadvisedby',
'ta',
'hasposition',
'publication',
'inphase',
'courselevel',
'yearsinprogram',
'projectmember',
])

## Background  configuration

In [4]:
bk = boostsrl.example_data('background')
bk = ['professor(+person).',
'student(+person).',
'advisedby(+person,+person).',
'advisedby(+person,-person).',
'advisedby(-person,+person).',
'tempadvisedby(+person,+person).',
'tempadvisedby(+person,-person).',
'tempadvisedby(-person,+person).',
'ta(+course,+person,+quarter).',
'ta(-course,+person,+quarter).',
'ta(+course,-person,+quarter).',
'ta(+course,+person,-quarter).',
'ta(-course,+person,-quarter).',
'ta(+course,-person,-quarter).',
'hasposition(+person,+faculty).',
'hasposition(+person,-faculty).',
'hasposition(-person,+faculty).',
'publication(+title,+person).',
'publication(+title,-person).',
'publication(-title,+person).',
'inphase(+person,+prequals).',
'inphase(+person,-prequals).',
'inphase(-person,+prequals).',
'courselevel(+course,+level).',
'courselevel(+course,-level).',
'courselevel(-course,+level).',
'yearsinprogram(+person,+year).',
'yearsinprogram(-person,+year).',
'yearsinprogram(+person,-year).',
'projectmember(+project,+person).',
'projectmember(+project,-person).',
'projectmember(-project,+person).']
#'sameproject(project, project).',
#'samecourse(course, course).',
#'sameperson(person, person).',]

background = boostsrl.modes(bk, [target], useStdLogicVariables=False, treeDepth=8, nodeSize=3, numOfClauses=8)

## Sampling examples

In [5]:
# shuffle all examples
random.shuffle(pos)
random.shuffle(neg)

neg = neg[:len(pos)] # balanced

import numpy as np
pos = np.array(pos)
neg = np.array(neg)
small_dataset_aucroc = []
complete_dataset_aucroc = []
revision_dataset_aucroc = []


# separate train and test
kf = KFold(n_splits=10)
for train_index, test_index in kf.split(pos):
    train_pos, test_pos = pos[train_index], pos[test_index]
    train_neg, test_neg = neg[train_index], neg[test_index]

    # learn from scratch in a small dataset
    s_train_pos = train_pos[:int(small_train_size*len(train_pos))]
    s_train_neg = train_neg[:int(small_train_size*len(train_neg))]

    # shuffle all train examples
    random.shuffle(train_pos)
    random.shuffle(train_neg)

    # train set used in revision and validation set
    r_train_pos = train_pos[int(validation_size*len(train_pos)):]
    r_train_neg = train_neg[int(validation_size*len(train_neg)):]
    validation_pos = train_pos[:int(validation_size*len(train_pos))]
    validation_neg = train_neg[:int(validation_size*len(train_neg))]

    print('Total examples')
    print('Positive examples: %s' % len(pos))
    print('Negative examples: %s' % len(neg))
    print('\n')
    print('Train and test sets from total')
    print('Train Positive examples: %s' % len(train_pos))
    print('Train Negative examples: %s' % len(train_neg))
    print('Test Positive examples: %s' % len(test_pos))
    print('Test Negative examples: %s' % len(test_neg))
    print('\n')
    print('Small dataset')
    print('Train Positive examples: %s' % len(s_train_pos))
    print('Train Negative examples: %s' % len(s_train_neg))
    print('\n')
    print('Revision theory train dataset and validation')
    print('Train Positive examples: %s' % len(r_train_pos))
    print('Train Negative examples: %s' % len(r_train_neg))
    print('Validation Positive examples: %s' % len(validation_pos))
    print('Validation Negative examples: %s' % len(validation_neg))
    
    # learning from small dataset
    delete_model_files()
    model = boostsrl.train(background, s_train_pos, s_train_neg, facts)
    learning_time = model.traintime()
    will = model.get_will_produced_tree()
    structured = model.get_structured_tree().copy()
    results = boostsrl.test(model, test_pos, test_neg, facts)
    inference_time = results.testtime()
    t_results = results.summarize_results()
    print('WILL-Produced Tree:')
    print_will_produced_tree(will)
    print('\n')
    print('Results:')
    print(t_results)
    print('\n')
    print('Total learning time: %s seconds' % learning_time)
    print('Total inference time: %s seconds' % inference_time)
    print('AUC ROC: %s' % t_results['AUC ROC'])
    print('\n')
    #print('Tree:')
    #model.tree(0, target, image=True)
    small_dataset_aucroc.append(t_results['AUC ROC'])
    
    # learning from complete dataset
    delete_model_files()
    model = boostsrl.train(background, train_pos, train_neg, facts)
    learning_time = model.traintime()
    will = model.get_will_produced_tree()
    #structured = model.get_structured_tree().copy()
    results = boostsrl.test(model, test_pos, test_neg, facts)
    inference_time = results.testtime()
    t_results = results.summarize_results()
    print('WILL-Produced Tree:')
    print_will_produced_tree(will)
    print('\n')
    print('Results:')
    print(t_results)
    print('\n')
    print('Total learning time: %s seconds' % learning_time)
    print('Total inference time: %s seconds' % inference_time)
    print('AUC ROC: %s' % t_results['AUC ROC'])
    print('\n')
    #print('Tree:')
    #model.tree(0, target, image=True)
    complete_dataset_aucroc.append(t_results['AUC ROC'])
    
    # theory revision
    total_revision_time = 0
    best_aucroc = 0
    best_structured = None

    print('Performing parameter learning')
    print('******************************************')
    delete_model_files()
    model = boostsrl.train(background, r_train_pos, r_train_neg, facts, refine=get_refine_file(structured))
    learning_time = model.traintime()
    will = model.get_will_produced_tree()
    structured = model.get_structured_tree().copy()
    results = boostsrl.test(model, validation_pos, validation_neg, facts)
    inference_time = results.testtime()
    t_results = results.summarize_results()
    total_revision_time += learning_time + inference_time
    print('WILL-Produced Tree:')
    print_will_produced_tree(will)
    print('\n')
    print('Results:')
    print(t_results)
    print('\n')
    print('Total learning time: %s seconds' % learning_time)
    print('Total inference time: %s seconds' % inference_time)
    print('AUC ROC: %s' % t_results['AUC ROC'])
    print('\n')
    #print('Tree:')
    #model.tree(0, target, image=True)

    best_aucroc = t_results['AUC ROC']
    best_structured = structured.copy()
    save_model_files()

    for i in range(max_revision_iterations):
        print('Refining iteration %s' % str(i+1))
        print('******************************************')
        found_better = False
        for candidate in get_cantidates(best_structured, revision_threshold):
            print('Refining node candidate')
            print('******************************************')
            delete_model_files()
            model = boostsrl.train(background, r_train_pos, r_train_neg, facts, refine=candidate)
            learning_time = model.traintime()
            will = model.get_will_produced_tree()
            results = boostsrl.test(model, validation_pos, validation_neg, facts)
            inference_time = results.testtime()
            t_results = results.summarize_results()
            total_revision_time += learning_time + inference_time
            print('WILL-Produced Tree:')
            print_will_produced_tree(will)
            print('\n')
            print('Results:')
            print(t_results)
            print('\n')
            print('Total learning time: %s seconds' % learning_time)
            print('Total inference time: %s seconds' % inference_time)
            print('AUC ROC: %s' % t_results['AUC ROC'])
            print('\n')
            #print('Tree:')
            #model.tree(0, target, image=True)
            print('******************************************')

            if t_results['AUC ROC'] > best_aucroc:
                found_better = True
                best_aucroc = t_results['AUC ROC']
                best_structured = model.get_structured_tree().copy()
                save_model_files()
        print('Best Tree AUC ROC so far: %s' % best_aucroc)
        print('******************************************\n')
        if found_better == False:
            break

    print('******************************************')
    delete_model_files()
    get_saved_model_files()
    delete_test_files()
    print('Total revision time: %s' % total_revision_time)
    print('Best validation AUC ROC: %s' % best_aucroc)
    will = model.get_will_produced_tree()
    results = boostsrl.test(model, test_pos, test_neg, facts)
    inference_time = results.testtime()
    t_results = results.summarize_results()
    print('WILL-Produced Tree:')
    print_will_produced_tree(will)
    print('\n')
    print('Results:')
    print(t_results)
    print('\n')
    print('Total inference time: %s seconds' % inference_time)
    print('AUC ROC: %s' % t_results['AUC ROC'])
    print('\n')
    #print('Tree:')
    #model.tree(0, target, image=True)
    revision_dataset_aucroc.append(t_results['AUC ROC'])

Total examples
Positive examples: 113
Negative examples: 113


Train and test sets from total
Train Positive examples: 101
Train Negative examples: 101
Test Positive examples: 12
Test Negative examples: 12


Small dataset
Train Positive examples: 30
Train Negative examples: 30


Revision theory train dataset and validation
Train Positive examples: 81
Train Negative examples: 81
Validation Positive examples: 20
Validation Negative examples: 20
WILL-Produced Tree:
% FOR advisedby(A, B):
%   if ( professor(B), student(A) )
%   then if ( publication(C, A) )
%   | then return 0.8581489350995123;  // std dev = 0,000, 16,000 (wgt'ed) examples reached here.  /* #pos=16 */
%   | else if ( ta(D, A, E) )
%   | | then return 0.8581489350995123;  // std dev = 0,000, 9,000 (wgt'ed) examples reached here.  /* #pos=9 */
%   | | else return 0.6914822684328455;  // std dev = 0,913, 6,000 (wgt'ed) examples reached here.  /* #neg=1 #pos=5 */
%   else return -0.1418510649004877;  // std dev = 2,36e-08, 29,

In [6]:
small_dataset_aucroc

[0.947917,
 0.847222,
 0.923611,
 1.0,
 0.917355,
 0.863636,
 0.995868,
 0.958678,
 0.92562,
 1.0]

In [7]:
complete_dataset_aucroc

[1.0,
 0.972222,
 0.920139,
 1.0,
 0.760331,
 0.694215,
 0.942149,
 0.876033,
 0.958678,
 1.0]

In [8]:
revision_dataset_aucroc

[0.951389,
 0.902778,
 0.972222,
 1.0,
 0.92562,
 0.896694,
 0.96281,
 0.917355,
 0.92562,
 1.0]

In [9]:
print('Small dataset AUC ROC: %s +/- %s' % ((np.array(small_dataset_aucroc)).mean(), (np.array(small_dataset_aucroc)).std()))
print('Complete dataset AUC ROC: %s +/- %s' % ((np.array(complete_dataset_aucroc)).mean(), (np.array(complete_dataset_aucroc)).std()))
print('Theory revision dataset AUC ROC: %s +/- %s' % ((np.array(revision_dataset_aucroc)).mean(), (np.array(revision_dataset_aucroc)).std()))


Small dataset AUC ROC: 0.9379907 +/- 0.05110753824838367
Complete dataset AUC ROC: 0.9123767 +/- 0.10086561769805408
Theory revision dataset AUC ROC: 0.9454488 +/- 0.03574719602374431
